# Event-based Simulation with SimPy

In [ ]:
import simpy as sp
import numpy as np
import scipy.stats

## SimPy Basics

SimPy has a few basic concepts to let you perform event-based simulation:
1. **Environment:** think of this as "the world" in which your simulation occurs
2. **Event:** an event that occurs in the environment -- it can be a happening, from time ticking to an action occurring
3. **Process:** described by Python generator functions -- they create events and `yield` (yeet) them out to the world
4. **Timeout:** a special time-based event, that describes how long it takes for an event to occur (assumed units)
5. **Resource:** an object representing a resource that can be acquired or released by Processes (gas station with limited fuel pumps)
6. **Container:** an object representing a resource that can be produced or consumed (like electrical power or apples)
7. **Store:** an object representing a store that can replenish or sell its items (dynamic quantities)

One thing that is key about event-based simulation is that, much like embedded systems, all processes are **interruptible** by priority, meaning that processes can be at different priorities (just like real life).

If you need to yield multiple events (actions) at a time, use the `&` operator.  If you need to yield at least one event (whichever event is closest), use the `|` operator.

### DMV customer example

Just to show everything in context, I'll modify the [Bank example](https://simpy.readthedocs.io/en/latest/examples/bank_renege.html) to the DMV:

In [ ]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def source(env, number, interval, counter, the_rng):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = the_rng.exponential(interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank, the_rng):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = the_rng.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = the_rng.exponential(time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))


# Setup and start the simulation
print('SIMULATION: DMV in all of its glory')
the_rng = np.random.default_rng(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

### Sample SimPy problem

Let's perform a simulation of four electric cars (EV, electric vehicles) trying to drive to and use a battery charging station. A car will perform three actions:
1. Driving to the EV charging station (accept the driving time as an input parameter).
2. Request/acquire a charging spot.
3. Charge the battery (accept the time it takes to charge the battery as an input parameter).

Define the charging station as only having two spots, so we have a likely chance of at least one car waiting for charging to complete.
Let's use a uniform distribution between 2 and 6 miles to model the distance each car has to drive in order to reach the EV charging station. Let's also assume that the cars can drive 35 miles per hour to reach the charging station.  Let's also use a uniform distribution to model the number of minutes that each car has to charge, from 30 minutes to 90 minutes.

LPT: Don't forget the random seed!

## Handling priorities and interrupts

Let's bump the EV charging simulation up now, to 10 cars, and pretend that a couple of the cars have made reservations online, so they can jump the queue (`simpy.PriorityResource`).  If the class wants, we could even say someone has a VIP card that can kick someone off of a pump (`simpy.PreemptiveResource`).

We could even use interrupts in a non-mean way, to indicate when a driver might have just gotten a text message and needs to leave and be somewhere instead.  Let's discuss how to model this event!

## Using a `class` to define more complex items that have multiple simultaneous processes

In the initialization, make sure to do two things:
1. Create a `<classname>_reactivate` member that is a generic `Event` that can be used to trigger interrupts.
2. Define each member function as having a `process` for each process that the complex item does.

Let's make our EV a bit fancier!  We can add the following:
1. We now keep track of our battery level.
2. We have a solar charger on our EV that automatically charges once we drive, and disengages on interrupt (if we drive).

Let's make a simulation where our driver will drive to the mall, shop for some random time, then come back and drive away (which should interrupt the solar charging process).

## Monitoring your SimPy simulation

There are three ways to go about monitoring a simulation, in order from easy to difficult:
1. Implement your own monitoring harness (basically, add input/output parameters for monitoring into your process functions)
2. Patching (creating a wrapper) for Simpy `Events` ([see this link](https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing))
3. Patching (creating a wrapper) for SimPy `Resources` ([see this link](https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#resource-usage))

# Using SimPy for real-time ECE problems

Turns out that SimPy also has a RealtimeEnvironment, which lets you synchronize events with wall-clock time.

This means that we can perform the following simulations:
1. Hardware-in-the-loop testing (VERY useful in electronics, embedded systems, and medical devices)
2. Testing that requires human interaction (energy, sustainability, etc.)

**Let's discuss:** what might be an event-based simulation problem in your specialization